In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Changing Check Settings for FD or CS

## Changing Settings for Inputs on a Specific Component

You can change the settings for the approximation schemes that will be used to compare with your component’s derivatives by calling the `set_check_partial_options` method.

```{eval-rst}
    .. automethod:: openmdao.core.component.Component.set_check_partial_options
        :noindex:
```

```{Note}
If you want to use method=”cs”, then you must also pass `force_alloc_complex`=True to setup. See the example below.
``` 

This allows custom tailoring of the approximation settings on a variable basis.

## Usage Examples

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src59", get_code("openmdao.test_suite.components.paraboloid_mat_vec.ParaboloidMatVec"), display=False)

:::{Admonition} `ParaboloidMatVec` class definition 
:class: dropdown

{glue:}`code_src59`
:::

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src60", get_code("openmdao.core.tests.test_check_derivs.ParaboloidTricky"), display=False)

:::{Admonition} `ParaboloidTricky` class definition 
:class: dropdown

{glue:}`code_src60`
:::

In [ ]:
import openmdao.api as om
from openmdao.core.tests.test_check_derivs import ParaboloidTricky
from openmdao.test_suite.components.paraboloid_mat_vec import ParaboloidMatVec

prob = om.Problem()

comp = prob.model.add_subsystem('comp', ParaboloidTricky())
prob.model.add_subsystem('comp2', ParaboloidMatVec())

prob.model.connect('comp.f_xy', 'comp2.x')

prob.set_solver_print(level=0)

comp.set_check_partial_options(wrt='*', step=1e-2)

prob.setup()
prob.run_model()

prob.check_partials(compact_print=True)

Here, we show how to set the method. In this case, we use complex step on TrickyParaboloid because the finite difference is less accurate.

---

```{Note}
You need to `set force_alloc_complex` to True during setup to utilize complex step during a check.
```

In [ ]:
import openmdao.api as om
from openmdao.core.tests.test_check_derivs import ParaboloidTricky
from openmdao.test_suite.components.paraboloid_mat_vec import ParaboloidMatVec

prob = om.Problem()

comp = prob.model.add_subsystem('comp', ParaboloidTricky())
prob.model.add_subsystem('comp2', ParaboloidMatVec())

prob.model.connect('comp.f_xy', 'comp2.x')

prob.set_solver_print(level=0)

comp.set_check_partial_options(wrt='*', method='cs')

prob.setup(force_alloc_complex=True)
prob.run_model()

prob.check_partials(compact_print=True)

## Directional Derivatives
You can also specify that an input or set of inputs be checked using a directional derivative. For vector inputs, this means that, instead of calculating the derivative with respect to each element of the array, we calculate the derivative with respect to a linear combination of all array indices. For finite difference and complex step, the step value is applied simultaneously to all elements of the vector. This is a much quicker check because it only requires a single execution of the component for the variable rather than one for each element of the array.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src61", get_code("openmdao.test_suite.components.array_comp.ArrayComp"), display=False)

:::{Admonition} `ArrayComp` class definition 
:class: dropdown

{glue:}`code_src61`
:::

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.array_comp import ArrayComp

prob = om.Problem()
model = prob.model
mycomp = model.add_subsystem('mycomp', ArrayComp(), promotes=['*'])

prob.setup()
prob.run_model()

data = prob.check_partials()

If your component is matrix-free and you request directional derivatives, then your reverse mode derivatives will be verified using the dot product test described [here](http://www.reproducibility.org/RSF/book/gee/ajt/paper_html/node20.html).

In [ ]:
import numpy as np

import openmdao.api as om

class ArrayCompMatrixFree(om.ExplicitComponent):

    def setup(self):

        J1 = np.array([[1.0, 3.0, -2.0, 7.0],
                        [6.0, 2.5, 2.0, 4.0],
                        [-1.0, 0.0, 8.0, 1.0],
                        [1.0, 4.0, -5.0, 6.0]])

        self.J1 = J1
        self.J2 = J1 * 3.3
        self.Jb = J1.T

        # Inputs
        self.add_input('x1', np.zeros([4]))
        self.add_input('x2', np.zeros([4]))
        self.add_input('bb', np.zeros([4]))

        # Outputs
        self.add_output('y1', np.zeros([4]))

        self.set_check_partial_options('*', directional=True)

    def compute(self, inputs, outputs):
        """
        Execution.
        """
        outputs['y1'] = self.J1.dot(inputs['x1']) + self.J2.dot(inputs['x2']) + self.Jb.dot(inputs['bb'])

    def compute_jacvec_product(self, inputs, dinputs, doutputs, mode):
        """Returns the product of the incoming vector with the Jacobian."""

        if mode == 'fwd':
            if 'x1' in dinputs:
                doutputs['y1'] += self.J1.dot(dinputs['x1'])
            if 'x2' in dinputs:
                doutputs['y1'] += self.J2.dot(dinputs['x2'])
            if 'bb' in dinputs:
                doutputs['y1'] += self.Jb.dot(dinputs['bb'])

        elif mode == 'rev':
            if 'x1' in dinputs:
                dinputs['x1'] += self.J1.T.dot(doutputs['y1'])
            if 'x2' in dinputs:
                dinputs['x2'] += self.J2.T.dot(doutputs['y1'])
            if 'bb' in dinputs:
                dinputs['bb'] += self.Jb.T.dot(doutputs['y1'])

prob = om.Problem()
model = prob.model
model.add_subsystem('mycomp', ArrayCompMatrixFree(), promotes=['*'])

prob.setup()
prob.run_model()

data = prob.check_partials()

## Changing Global Settings For Whole Model

You can change the settings globally for all approximations used for all components. This is done by passing in a value for any of the following arguments:

| Name      | Description                                                                                    |
|-----------|------------------------------------------------------------------------------------------------|
| method    | Method for check: “fd” for finite difference, “cs” for complex step.                           |
| form      | Finite difference form for check, can be “forward”, “central”, or backward.                    |
| step      | Step size for finite difference check.                                                         |
| step_calc | Type of step calculation for check, can be “abs” for absolute (default) or “rel” for relative. |


```{Note}
The global check options take precedence over the ones defined on a component.
``` 

## Usage Examples

In [ ]:
import openmdao.api as om
from openmdao.core.tests.test_check_derivs import ParaboloidTricky
from openmdao.test_suite.components.paraboloid_mat_vec import ParaboloidMatVec

prob = om.Problem()

prob.model.add_subsystem('comp', ParaboloidTricky())
prob.model.add_subsystem('comp2', ParaboloidMatVec())

prob.model.connect('comp.f_xy', 'comp2.x')

prob.set_solver_print(level=0)

prob.setup()
prob.run_model()

prob.check_partials(step=1e-2, compact_print=True)

```{Note}
You need to set `force_alloc_complex` to True during setup to utilize complex step during a check.
```

In [ ]:
import openmdao.api as om
from openmdao.core.tests.test_check_derivs import ParaboloidTricky
from openmdao.test_suite.components.paraboloid_mat_vec import ParaboloidMatVec

prob = om.Problem()

prob.model.add_subsystem('comp', ParaboloidTricky())
prob.model.add_subsystem('comp2', ParaboloidMatVec())

prob.model.connect('comp.f_xy', 'comp2.x')

prob.set_solver_print(level=0)

prob.setup(force_alloc_complex=True)
prob.run_model()

prob.check_partials(method='cs', compact_print=True)

---

In [ ]:
import openmdao.api as om
from openmdao.core.tests.test_check_derivs import ParaboloidTricky
from openmdao.test_suite.components.paraboloid_mat_vec import ParaboloidMatVec

prob = om.Problem()

prob.model.add_subsystem('comp', ParaboloidTricky())
prob.model.add_subsystem('comp2', ParaboloidMatVec())

prob.model.connect('comp.f_xy', 'comp2.x')

prob.set_solver_print(level=0)

prob.setup()
prob.run_model()

prob.check_partials(form='central', compact_print=True)

---

In [ ]:
import openmdao.api as om
from openmdao.core.tests.test_check_derivs import ParaboloidTricky

prob = om.Problem()

prob.model.add_subsystem('comp', ParaboloidTricky())

prob.set_solver_print(level=0)

prob.setup()
prob.run_model()

prob.check_partials(step_calc='rel', compact_print=True)